In [ ]:
##%%capture
%pip uninstall jwt
%pip install PyJWT -U
%pip install yandex-chain

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
%%capture
import requests
import json
import time
import jwt
import os

In [ ]:
service_account_json = json.loads(os.environ["service_account_json"])
service_account_id = service_account_json["service_account_id"]
key_id = service_account_json["id"]
private_key = service_account_json["private_key"]

In [ ]:
now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
token = x['iamToken']

Проверяем, все ли работает и все ли загрузилось.

In [ ]:
from yandex_chain import YandexLLM, YandexGPTModel

LLM = YandexLLM(model=YandexGPTModel.Pro, folder_id=folder_id, iam_token=token)
print(LLM._modelUri)
print(LLM("Buenos dias. Que tal?"))

gpt://b1gnor6f8dtj2useoeoc/yandexgpt/latest


/tmp/ipykernel_2135/1426869443.py:5: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  print(LLM("Buenos dias. Que tal?"))


Buenos días. ¡Todo bien, gracias! ¿Y usted? (или просто «¡Bien!»).


Инициализируем модель.

In [ ]:
LLM = YandexLLM(model=YandexGPTModel.Pro, folder_id=folder_id, iam_token=token, temperature=0.0)

Посмотрим, как модель справится с такой задачей, как планирование путешествия. Будем уточнять подводку с каждым шагом.

То, что мы бы хотели увидеть, это:
- название маршрута/путешествия
- вид транспорта (2 из 3)
- маршрут при выбранном виде транспорта
- время в пути
- рекомендованные остановки и интересные места в них

# zero-shot

In [ ]:
def make_request(prompt, text):
    return f"{prompt}\n\nМАРШРУТ: {text}"

first_prompt = '''Ты помощник по планированию путешествий. Твоя задача -- спланировать путешествие из точки A в точку B.

Маршрут будет написан после слова МАРШРУТ. Ты должна написать план для путешествия для данного маршрута после слова ПЛАН.'''

In [ ]:
text = "МОСКВА - САНКТ-ПЕТЕРБУРГ"

request = make_request(first_prompt, text)
print(LLM(request))

**Маршрут:** Москва — Санкт-Петербург.

**План:**
1. Выберите способ передвижения: поезд, автобус, автомобиль или самолёт. Каждый из этих вариантов имеет свои преимущества и недостатки, поэтому выбор зависит от ваших предпочтений и бюджета.
2. Если вы выбрали поезд, то заранее купите билеты на сайте РЖД или в кассах вокзала. Это позволит вам выбрать удобное время отправления и место в вагоне.
3. Если вы путешествуете на автобусе, также заранее приобретите билеты на сайтах компаний-перевозчиков или на автовокзале. Проверьте расписание рейсов и выберите наиболее подходящий вариант.
4. При поездке на автомобиле спланируйте маршрут с учётом пробок и загруженности дорог. Заправьте полный бак бензина и проверьте техническое состояние автомобиля перед отправлением.
5. Если вы летите на самолёте, забронируйте билеты на рейс одной из авиакомпаний, выполняющих рейсы между Москвой и Санкт-Петербургом. Учтите время на регистрацию и прохождение досмотра в аэропорту.
6. По прибытии в пункт назначени

План, сгенерированный моделью, пока что сильно отличается от того, что мы бы хотели увидеть (но мы еще и не сообщили модели этого). Ответ модели весьма общий и неспецифичный. Попробуем изменить подводку.

In [ ]:
second_prompt = '''Ты помощник по планированию путешествий. Твоя задача -- спланировать путешествие из точки A в точку B, выбрав оптимальный маршурут и расчитав ориентировочное время в пути, так же можешь добавить в план что-то свое.

Маршрут будет написан после слова МАРШРУТ. Ты должна написать план для путешествия для данного маршрута после слова ПЛАН.'''
text = "МОСКВА - САНКТ-ПЕТЕРБУРГ"

request = make_request(second_prompt, text)
print(LLM(request))

**Маршрут:** Москва — Санкт-Петербург.

**Виды транспорта:** поезд или автомобиль.

**Время в пути:**
* на поезде: от 3,5 до 10 часов (в зависимости от типа поезда и времени отправления);
* на автомобиле: около 6–8 часов (без учёта пробок и остановок).

**План путешествия:**
1. Выберите способ передвижения: поезд или автомобиль, учитывая время отправления, расписание и бюджет.
2. Если вы путешествуете на поезде, заранее купите билеты, чтобы выбрать удобные места и вагон.
3. Если вы едете на машине, рассчитайте маршрут с учётом пробок и возможных остановок. Проверьте состояние автомобиля и наличие необходимых документов.
4. Возьмите с собой всё необходимое: документы, деньги, еду, напитки, одежду по погоде.
5. По прибытии в Санкт-Петербург можно сразу отправиться на прогулку по городу или отдохнуть после дороги.


Начинаем приближаться к тому, чего хотим. Продолжаем редактировать подводку.

In [ ]:
second_prompt_v2 = '''Ты помощник по планированию путешествий. Твоя задача -- спланировать путешествие из точки A в точку B, выбрав оптимальный маршурут и расчитав ориентировочное время в пути, так же можешь добавить в план что-то свое.
Путешествие планируется на самолете, поезде или автомобиле. Используй сторонние сервисы для более конкретного подсчета времени в пути. Так же добавь возможные остановки.

Маршрут будет написан после слова МАРШРУТ. Ты должна написать план для путешествия для данного маршрута после слова ПЛАН.'''
text = "МОСКВА - САНКТ-ПЕТЕРБУРГ"

request = make_request(second_prompt_v2, text)
print(LLM(request))

**Маршрут:** Москва — Санкт-Петербург.

**Оптимальный способ передвижения:** поезд или автомобиль (в зависимости от предпочтений и бюджета).

1. **Поезд.**

* **Время в пути:** 3,5–4 часа на скоростном поезде «Сапсан».
* **Остановки:** по желанию можно сделать остановку в Твери или других городах по пути следования поезда.

2. **Автомобиль.**

* **Расстояние:** около 630 км.
* **Время в пути:** примерно 7–8 часов без остановок.
* **Возможные остановки:** Новгород Великий, Вышний Волочёк, Тверь.

**План путешествия:**

1. Выберите способ передвижения: поезд или автомобиль.
2. Если вы выбрали поезд, то купите билеты заранее на сайте РЖД или в кассах вокзала.
3. Если вы путешествуете на автомобиле, то проверьте техническое состояние машины и наличие всех необходимых документов.
4. При путешествии на автомобиле спланируйте маршрут с учётом возможных остановок для отдыха и еды.
5. В любом случае возьмите с собой необходимые вещи: документы, деньги, телефон, еду и напитки.
6. Наслаждайтесь п

Добавили возможные остановки. Сторонние сервисы модель не привела. В следующем варианте подводки попросим ее это сделать (точнее, сделаем это через один вариант подводки. Пока что приводим возможные сторонние сервисы).

In [ ]:
second_prompt_v3 = '''Ты помощник по планированию путешествий. Твоя задача -- спланировать путешествие из точки A в точку B, выбрав оптимальный маршурут и расчитав ориентировочное время в пути, так же можешь добавить в план что-то свое.
Путешествие планируется на самолете, поезде или автомобиле. Используй сторонние сервисы для более конкретного подсчета времени в пути. Так же добавь возможные остановки.
Сторонние сервисы, которые можно использовать: rasp.yandex.ru, aviasales.ru, tutu.ru, travel.yandex.ru, rzd.ru, yandex.maps.ru.

Маршрут будет написан после слова МАРШРУТ. Ты должна написать план для путешествия для данного маршрута после слова ПЛАН.'''
text = "МОСКВА - САНКТ-ПЕТЕРБУРГ"

request = make_request(second_prompt_v3, text)
print(LLM(request))

**Маршрут:**
1. Москва — Санкт-Петербург на скоростном поезде «Сапсан».
2. Время в пути составит примерно 3,5–4 часа.

**План:**
1. Выбрать и купить билеты на поезд на сайте РЖД или других сервисах (tutu.ru).
2. Подготовиться к поездке: упаковать вещи, взять документы и необходимые предметы.
3. Прибыть на вокзал за 30 минут до отправления поезда.
4. Провести время в поезде: почитать книгу, послушать музыку или посмотреть фильм.
5. По прибытии в Санкт-Петербург можно сразу отправиться по делам или отдохнуть после поездки.


Какой-то совсем невнятный ответ. Видимо надо указать, что нужно БРАТЬ информацию из сервисов, а не просто их указывать.

In [ ]:
second_prompt_v4 = '''Ты помощник по планированию путешествий. Твоя задача -- спланировать путешествие из точки A в точку B, выбрав оптимальный маршурут и расчитав ориентировочное время в пути, так же можешь добавить в план что-то свое.
Путешествие планируется на самолете, поезде или автомобиле. Используй сторонние сервисы для более конкретного подсчета времени в пути. Так же добавь возможные остановки.
Сторонние сервисы, которые можно использовать: rasp.yandex.ru, aviasales.ru, tutu.ru, travel.yandex.ru, rzd.ru, yandex.maps.ru.
Сделай план развернутым, приводи ссылки на сервисы, когда ты берешь из них информацию.
Маршрут будет написан после слова МАРШРУТ. Ты должна написать план для путешествия для данного маршрута после слова ПЛАН.'''
text = "МОСКВА - САНКТ-ПЕТЕРБУРГ"

request = make_request(second_prompt_v4, text)
print(LLM(request))

**Маршрут:** Москва — Санкт-Петербург.

**Виды транспорта:** поезд, самолёт, автомобиль.

1. **Самолёт.** Самый быстрый способ добраться из Москвы в Санкт-Петербург — это самолёт. Время в пути составит примерно 1 час 20 минут. Стоимость билетов зависит от даты вылета и авиакомпании. Для поиска и покупки билетов можно воспользоваться сервисом aviasales.ru.

2. **Поезд.** Если вы предпочитаете путешествовать по железной дороге, то можно выбрать поезд. Время в пути на «Сапсане» составит около 4 часов. Билеты можно купить на сайте rzd.ru или в кассах железнодорожных вокзалов.

3. **Автомобиль.** Путешествие на автомобиле может быть интересным и удобным, но оно также может занять больше времени. Расстояние между Москвой и Санкт-Петербургом составляет примерно 630 км. Время в пути без остановок может составить около 8–9 часов. В зависимости от дорожной ситуации и скорости движения время может варьироваться.

Для расчёта ориентировочного времени в пути с учётом остановок можно использовать се

Стало лучше чем в последнем ответе, но хуже, чем в ответах до него. Возможно, стоит напомнить, что еще нужно упомянуть в плане. Так же попросим модель порекомендовать достопримечательности в точке прибытия.

In [ ]:
second_prompt_v5 = '''Ты помощник по планированию путешествий. Твоя задача -- спланировать путешествие из точки A в точку B, выбрав оптимальный маршурут и расчитав ориентировочное время в пути, так же можешь добавить в план что-то свое.
Путешествие планируется на самолете, поезде или автомобиле. Используй сторонние сервисы для более конкретного подсчета времени в пути. Так же добавь возможные остановки.
Сторонние сервисы, которые можно использовать: rasp.yandex.ru, aviasales.ru, tutu.ru, travel.yandex.ru, rzd.ru, yandex.maps.ru.
Сделай план развернутым, приводи ссылки на сервисы, когда ты берешь из них информацию.
Не забудь про остановки и расстояние. Можешь указать, чем конкретно можно заняться в точке B.

Маршрут будет написан после слова МАРШРУТ. Ты должна написать план для путешествия для данного маршрута после слова ПЛАН.'''
text = "МОСКВА - САНКТ-ПЕТЕРБУРГ"

request = make_request(second_prompt_v5, text)
print(LLM(request))

**Маршрут:** Москва — Санкт-Петербург.

**Вид транспорта:** поезд или автомобиль (на выбор).

1. **Поезд.**

    * **Время в пути:** около 4–8 часов, в зависимости от типа поезда и расписания.
    * **Сторонний сервис для покупки билетов:** tutu.ru.
    * **Возможные остановки:** по пути следования поезда могут быть остановки в других городах, например, Тверь или Бологое. Можно рассмотреть возможность остановки на несколько часов для отдыха или прогулки по городу.

2. **Автомобиль.**

   * **Расстояние:** примерно 635 км.
   * **Ориентировочное время в пути без остановок:** 7–9 часов.
   * **Сторонние сервисы для планирования маршрута:** Яндекс Карты (yandex.maps.ru), Яндекс Расписания (rasp.yandex.ru).
   * **Возможные остановки:** можно сделать остановки для отдыха и перекуса на заправках или в кафе по пути. Также можно остановиться в Твери или Великом Новгороде для короткой прогулки и осмотра достопримечательностей.

**План путешествия:**

1. Выберите вид транспорта: поезд или автом

Стало хорошо, но моментами не по делу. Исправим.

In [ ]:
second_prompt_v6 = '''Ты помощник по планированию путешествий. Твоя задача -- спланировать путешествие из точки A в точку B, выбрав оптимальный маршурут и расчитав ориентировочное время в пути, так же можешь добавить в план что-то свое.
Путешествие планируется на самолете, поезде или автомобиле. Выбери два оптимальных вида транспорта и расскаажи про них. Используй сторонние сервисы для более конкретного подсчета времени в пути. Так же добавь возможные остановки.
Сторонние сервисы, которые можно использовать: rasp.yandex.ru, aviasales.ru, tutu.ru, travel.yandex.ru, rzd.ru, yandex.maps.ru.
Сделай план кратким и по делу, приводи ссылки на сервисы, когда ты берешь из них информацию.
Не забудь про остановки и расстояние. Можешь указать, чем конкретно можно заняться в точке B.

Маршрут будет написан после слова МАРШРУТ. Ты должна написать план для путешествия для данного маршрута после слова ПЛАН.'''
text = "МОСКВА - САНКТ-ПЕТЕРБУРГ"

request = make_request(second_prompt_v6, text)
print(LLM(request))

**Оптимальные виды транспорта:** поезд и автомобиль.

**1. Поезд.**

Время в пути: около 4–8 часов в зависимости от типа поезда и времени отправления.

Преимущества: комфорт, возможность отдохнуть во время поездки, не нужно беспокоиться о парковке и пробках.

Сервисы для покупки билетов: tutu.ru, rzd.ru.

**2. Автомобиль.**

Расстояние: примерно 630–700 км в зависимости от маршрута.

Время в пути: 8–10 часов без остановок.

Преимущества: гибкость маршрута, возможность остановиться по пути, чтобы отдохнуть или перекусить.

Возможные остановки: Тверь, Новгород, Валдай.

Сервис для планирования маршрута: yandex.maps.ru.

В Санкт-Петербурге можно посетить Эрмитаж, Русский музей, Петропавловскую крепость, прогуляться по Невскому проспекту, насладиться белыми ночами.

**МАРШРУТ:** Москва — Санкт-Петербург.

**ПЛАН:**

1. Выбрать подходящий поезд на сервисе tutu.ru или rzd.ru и купить билеты.
2. Собрать вещи и подготовиться к поездке.
3. Отправиться на вокзал и сесть на поезд.
4. Во время пое

Это ответ на финальный вариант подводки в зеро-шоте. В целом, он меня устраивает. Однако, во фью-шоте в примерах я не буду выписывать план, написанный моделью в конце ответа. Кажется, он выглядит громоздко и избыточно.

# few-shot + легкое модифицирование инструкции

In [ ]:
def make_request(prompt, text, samples=None):
    samples = samples or []
    answer = prompt
    for sample in samples:
        answer += "\n\nМАРШРУТ: {source}\nПЛАН: {target}".format(**sample)
    answer += f"\n\nМАРШРУТ: {text}"
    return answer

samples = [
    {
        "source": "РЯЗАНЬ - КАЗАНЬ",
        "target": '''Краткий план путешествия из Рязани в Казань

1. Автомобиль
Маршрут: Рязань - Владимир - Нижний Новгород - Казань
Время в пути: ~12-14 часов
Остановки:
- Владимир: Успенский собор, Золотые ворота.
- Нижний Новгород: Большая Покровская улица, Кремль.

2. Поезд
Маршрут: Рязань - Москва - Казань
Время в пути: ~10-12 часов
Остановки:
- Москва: Красная площадь, музеи, парки.

Рекомендации
- Автомобиль: Используйте Яндекс.Карты или Google Maps.
- Поезд: Проверьте расписание и цены на rzd.ru.

Чем заняться в Казани
- Казанский Кремль: мечеть Кул-Шариф, Благовещенский собор.
- Улица Баумана: прогулки, магазины, кафе.
- Озеро Кабан: отдых на природе.
- Национальный музей Республики Татарстан: история и культура региона.
- Аквапарк "Ривьера": развлечения для всей семьи.
- Старо-Татарская слобода: исторический район с музеями.'''
    },
    {
        "source": "САМАРА - ТЮМЕНЬ",
        "target": '''Краткий план путешествия из Самары в Тюмень
1. Автомобиль
Маршрут: Самара - Уфа - Челябинск - Тюмень
Время в пути: ~18-20 часов
Остановки:
- Уфа: посещение Монумента Дружбы, прогулка по городу.
- Челябинск: парк Гагарина, краеведческий музей.

2. Поезд
Маршрут: Самара - Екатеринбург - Тюмень
Время в пути: ~14-16 часов
Остановки:
- Екатеринбург: плотина Городского пруда, Храм-на-Крови, музеи.

Рекомендации
- Автомобиль: Используйте Яндекс.Карты или Google Maps.
- Поезд: Проверьте расписание и цены на rzd.ru.

Чем заняться в Тюмени
- Набережная реки Тура: прогулки, виды на город.
- Исторический парк "Россия — Моя история": экспозиции об истории России.
- Троицкий монастырь: духовная и историческая достопримечательность.
- термальные источники: расслабление и отдых.
- Сквер Сибирских Кошек: уютное место для отдыха.'''
    },
    {
        "source": "БАРНАУЛ - БЕЛОКУРИХА",
        "target": '''Краткий план путешествия из Барнаула в Белокуриху
1. Автомобиль
Маршрут: Барнаул - Бийск - Белокуриха
Время в пути: ~3-4 часа
Остановки:
- Бийск: посещение Успенского кафедрального собора, краеведческого музея.

2. Автобус
Маршрут: Барнаул - Белокуриха
Время в пути: ~4-5 часов
Остановки:
- По пути можно наслаждаться живописными видами Алтайского края.

Рекомендации
- Автомобиль: Используйте Яндекс.Карты или Google Maps для навигации.
- Автобус: Проверьте расписание на автовокзале Барнаула или онлайн.

Чем заняться в Белокурихе
- Прогулки по курортной зоне: чистый воздух и красивые пейзажи.
- Санатории и спа: лечение и отдых в одном из многочисленных санаториев.
- Гора Церковка: подъем на гору с панорамными видами.
- Минеральные источники: питьевое лечение и купание.
- Аквапарк Беловодье: развлечения для всей семьи.
- Музей истории курорта: узнайте больше о развитии региона.
'''
    }
]
text = "МОСКВА - САНКТ-ПЕТЕРБУРГ"

request = make_request(second_prompt_v6, text, samples)
print(LLM(request))

**Маршрут: Москва — Санкт-Петербург**

1. Поезд
Маршрут: Москва — Санкт-Петербург
Время в пути: 3,5–5 часов (в зависимости от типа поезда и количества остановок)

2. Автомобиль
Маршрут: Москва — Тверь — Великий Новгород — Санкт-Петербург (для более длительного путешествия)
Время в пути: ~10–12 часов
Остановки:
* Тверь: Императорский путевой дворец, прогулка по городу.
* Великий Новгород: Новгородский кремль, Ярославово дворище.

Рекомендации:
* Поезд: проверьте расписание и цены на rzd.ru.
* Автомобиль: используйте Яндекс.Карты или Google Maps для навигации.

Чем заняться в Санкт-Петербурге:
* Эрмитаж: один из крупнейших музеев мира.
* Петропавловская крепость: исторический центр города.
* Невский проспект: прогулки, магазины, кафе.
* Белые ночи: наблюдение за уникальным природным явлением.
* Мариинский театр: посещение спектаклей.
* Летний сад: отдых на природе.


Здесь мы дали три примера плана путешествия модели. Она успешно справилась с задачей после предоставления их нами. В качестве "вишенки на торте" попросим добавить в ответ интересный факт или немного "отсебятины".

In [ ]:
third_prompt = '''Ты помощник по планированию путешествий. Твоя задача -- спланировать путешествие из точки A в точку B, выбрав оптимальный маршурут и расчитав ориентировочное время в пути, так же можешь добавить в план что-то свое.
Путешествие планируется на самолете, поезде или автомобиле. Выбери два оптимальных вида транспорта и расскаажи про них. Используй сторонние сервисы для более конкретного подсчета времени в пути. Так же добавь возможные остановки.
Сторонние сервисы, которые можно использовать: rasp.yandex.ru, aviasales.ru, tutu.ru, travel.yandex.ru, rzd.ru, yandex.maps.ru.
Сделай план кратким и по делу, приводи ссылки на сервисы, когда ты берешь из них информацию.
Не забудь про остановки и расстояние. Можешь указать, чем конкретно можно заняться в точке B.
ОБЯЗАТЕЛЬНО добавь что-то свое: интересный факт о точке B или A, пожелание путешественнику или что-то на свой вкус.

Маршрут будет написан после слова МАРШРУТ. Ты должна написать план для путешествия для данного маршрута после слова ПЛАН.'''
text = "МОСКВА - САНКТ-ПЕТЕРБУРГ"

request = make_request(third_prompt, text, samples)
print(LLM(request))

**МАРШРУТ: Москва — Санкт-Петербург**

**ПЛАН:**
1. Поезд
Маршрут: Москва — Санкт-Петербург
Время в пути: ~8 часов

Рекомендации:
* Проверьте расписание и цены на rzd.ru.

Чем заняться в Санкт-Петербурге:
* Прогуляться по Невскому проспекту.
* Посетить Эрмитаж, Русский музей, Кунсткамеру.
* Сходить в Мариинский театр или Александринский театр.
* Полюбоваться разводными мостами.
* Попробовать пышки и корюшку.

2. Автомобиль
Маршрут: Москва — Тверь — Великий Новгород — Санкт-Петербург
Время в пути: ~10–12 часов
Остановки:
* Тверь: Императорский путевой дворец, прогулка по городу.
* Великий Новгород: Новгородский кремль, Ярославово дворище.

Рекомендации:
* Используйте Яндекс.Карты или Google Maps для навигации.

Интересный факт: между Москвой и Санкт-Петербургом находится множество городов Золотого кольца России, которые можно посетить по пути. Это позволит сделать путешествие более разнообразным и интересным.


В итоге мы получили вот такой ответ. Кажется, достаточно приемлемый. Полезно, пошагово и по делу.